<a href="https://colab.research.google.com/github/minoli-ford/FashionHub-chatbot/blob/main/ehub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import nltk
import json
import string
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, GlobalMaxPooling1D, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt

In [ ]:
#Sample questions and responses for the chatbot
%%writefile content.json
{
  "intent": [
    {
      "tag": "greeting",
      "input": ["hello", "hi", "nice to meet you", "any one here", "hi", "hey there", "hey", "hi there"],
      "responses": [
        "Hi, welcome to the Ehub",
        "Hey, do you need help?",
        "Welcome, how may I help you",
        "Hi, What do you want?",
        "Hello my friend, How can I help you"
      ]
    },
    {
      "tag": "goodbye",
      "input": ["Thank you", "Thanks", "Thanks for the info", "bye", "byr", "Okay bye", "Goodbye", "see you later", "i will catch you later"],
      "responses": [
        "Okay, Bye",
        "Have a nice day",
        "Goodbye",
        "Nice conversation, bye",
        "see you later",
        "Take care",
        "Yeh sure, will talk to you later"
      ]
    },
    {
      "tag": "conversation",
      "input": [
        "What time does the store open?",
        "Do you offer free shipping?",
        "How can I return a product?",
        "What payment methods do you accept?",
        "Tell me about your products."
      ],
      "responses": [
        "The store opens at 9:00 AM.",
        "Yes, we offer free shipping on orders over $50.",
        "To return a product, please contact our customer support.",
        "We accept credit cards, PayPal, and Apple Pay.",
        "We have a wide range of products, including electronics, clothing, and accessories."
      ]
    },
    {
      "tag": "product_inquiry",
      "input": [
        "show me the products",
        "what products do you have?",
        "can you show me some products?",
        "I want to see the available items",
        "display products"
      ],
      "responses": [
        "Sure, here are some of our products",
        "We have a wide range of products available. How can I assist you in finding one?",
        "Certainly, let me show you some of the amazing products we have!",
        "Sure, what type of products are you interested in?",
        "We have a great collection of products, want to take a look?"
      ]
    },
    {
      "tag": "purchase_help",
      "input": [
        "how do I place an order?",
        "can you help me with the checkout process?",
        "I need assistance with making a purchase",
        "how can I buy products?",
        "I want to place an order"
      ],
      "responses": [
        "Of course! To place an order, just follow these steps",
        "I'd be glad to help you with the checkout process. Here's a step-by-step guide",
        "No worries, I'll guide you through the purchase process. Please follow these instructions",
        "Sure, let me assist you with making a purchase. Here's what you need to do",
        "Placing an order is easy! Follow these steps"
      ]
    }
  ]
}





Writing content.json


In [ ]:
#importing the dataset
with open('content.json') as content:
    data1 = json.load(content)

In [ ]:
#getting all the data to list
tag = []
inputs = []
responses = {}
for intent in data1['intent']:
    responses[intent['tag']] = intent['responses']
    for lines in intent['input']:
        inputs.append(lines)
        tag.append(intent['tag'])

In [ ]:
#converting to dataframe
data = pd.DataFrame({"input": inputs, "tag": tag})


In [ ]:
#printing the data
data

,input,tag
0,hello,greeting
1,hi,greeting
2,nice to meet you,greeting
3,any one here,greeting
4,hi,greeting
5,hey there,greeting
6,hey,greeting
7,hi there,greeting
8,Thank you,goodbye
9,Thanks,goodbye


In [ ]:
#Pre-Processsing - Removing punctuatuins
import string
data['input'] = data['input'].apply(lambda wrd: [ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['input'] = data['input'].apply(lambda wrd: ''.join(wrd))
data

,input,tag
0,hello,greeting
1,hi,greeting
2,nice to meet you,greeting
3,any one here,greeting
4,hi,greeting
5,hey there,greeting
6,hey,greeting
7,hi there,greeting
8,thank you,goodbye
9,thanks,goodbye


In [ ]:
# Tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['input'])
train = tokenizer.texts_to_sequences(data['input'])

# Apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

# Encoding the output
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tag'])

In [ ]:
input_shape = x_train.shape[1]
print("Input shape:", input_shape)

Input shape: 8


In [ ]:
#define vocabulary
vocabulary = len(tokenizer.word_index)
print("Number of unique words:", vocabulary)
output_length = le.classes_.shape[0]
print("Output length:", output_length)

Number of unique words: 66
Output length: 5


In [ ]:
#creating the model

i = tf.keras.layers.Input(shape=(input_shape,))
x = tf.keras.layers.LSTM(10, return_sequences=True)(i)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(output_length, activation="softmax")(x)
model = tf.keras.models.Model(i, x)

ValueError: ignored

In [ ]:
#compiling the model
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])


In [ ]:
#train the model
train = model.fit(x_train_tfidf, y_train, epochs=200)


Epoch 1/200


ValueError: ignored

In [ ]:
#plottin model accuracy
plt.plot(train.history['accuracy'], label='Training set accuracy')
plt.plot(train.history['loss'], label='Training set loss')
plt.legend()

In [ ]:
import random

while True:
    texts_p = []
    prediction_input = input('You: ')

    # Removing punctuations and converting to lowercase
    prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
    prediction_input = ''.join(prediction_input)
    texts_p.append(prediction_input)

    # Tokenizing and padding
    prediction_input = tokenizer.texts_to_sequences(texts_p)
    prediction_input = np.array(prediction_input).reshape(-1)
    prediction_input = pad_sequences([prediction_input], input_shape)

    # Getting output
    output = model.predict(prediction_input)
    output = output.argmax()

    # Finding the right tag and predicting
    response_tag = le.inverse_transform([output])[0]
    print("fashionhub:", random.choice(responses[response_tag]))
    if response_tag == "goodbye":
        break